# Approach 2a : LULC using Google Dynamic & Landsat

 We utilized a total of 23 annotated images from the Landsat series: 12 from Landsat 8, 5 from Landsat 7, and 6 from Landsat 9 (20 for training, 3 for validation). Each image contained 21,459 pixels at a 30-meter resolution within the region of interest. These images were annotated using the Dynamic Land Cover dataset as a reference, with labels corresponding to the acquisition dates of the Landsat images. The use of Sentinel-2 data for this model is not relevant, as the DynamicWorld Land Cover dataset is already available for each Sentinel-2 observation.

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---


In [ ]:
import ee, geemap,eemont
import pandas as pd

import sys
sys.path.append('../preprocessing')
import landsat_preprocessing
import s2_preprocessing

import sys
sys.path.append('../wetlands_detection')
import wetlands_unsupervised_clustering

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# Preprocessing

In [2]:
def filter_col(col, roi, band, thresh):
    
    col = col.map(lambda image : image.clip(roi))

    def count_pixels(image,roi): 
        pixel_count = image.select(band).reduceRegion(
            reducer= ee.Reducer.count(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        ).get(band)
        return image.set('pixel_count', pixel_count)

    nb_pixels_ts = col.map(lambda image: count_pixels(image, roi))

    # Get the image with the maximum pixel count
    max_pixel_count_image = nb_pixels_ts.sort('pixel_count', False).first()
    ref_img_pixel_count = max_pixel_count_image.get('pixel_count').getInfo()
    pixel_count_threshold = ref_img_pixel_count * thresh

    # Filter the collection based on the pixel count threshold
    filtered_col = nb_pixels_ts.filter(ee.Filter.gte('pixel_count', pixel_count_threshold))

    return filtered_col

def get_gd_cloud_free_col(roi, thresh):
    
    gd = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1").filterBounds(roi)
    gd = filter_col(gd,roi,'label',thresh)
    
    return gd

def add_dates(image):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    return image.set({'date': date})

# Join collections 

In [3]:
def find_matching_images(l_collection, gd_collection):
    
    def find_matching_image(l_image):
        l_date = ee.Date(l_image.get('system:time_start'))
        matched_gd_image = gd_collection.filterDate(l_date, l_date.advance(1, 'day')).first()
        return ee.Algorithms.If(
            matched_gd_image,
            l_image.addBands(matched_gd_image.select('label')),
            None
        )

    matched_images = l_collection.map(find_matching_image, dropNulls=True)
    return ee.ImageCollection(matched_images)

roi = ee.Geometry.Polygon([[[17.204933, 60.402663], [17.204933, 60.455525], [17.2645, 60.455525], [17.2645, 60.402663], [17.204933, 60.402663]]])

l8 = landsat_preprocessing.get_l8_cloud_free_col(roi, ['NDVI', 'MNDWI', 'NDBI', 'NDMI'], 0.95)
l9 = landsat_preprocessing.get_l9_cloud_free_col(roi, ['NDVI', 'MNDWI', 'NDBI','NDMI'], 0.95)
l7 = landsat_preprocessing.get_l7_cloud_free_col(roi, ['NDVI', 'MNDWI', 'NDBI','NDMI'], 0.95)
gd = get_gd_cloud_free_col(roi, 0.95)

# Appliquer la fonction à chaque image dans la collection
gd = gd.map(lambda image : image.reproject(crs = (l8.first()).projection(),scale = 30))

l8Paired = ee.ImageCollection(find_matching_images(l8, gd))
l9Paired = ee.ImageCollection(find_matching_images(l9, gd))
l7Paired = ee.ImageCollection(find_matching_images(l7, gd))

lPaired = l8Paired.merge(l7Paired).merge(l9Paired)

display(lPaired)

# Prepare training data

In [4]:
def add_elev_slope(image): 

    jax_dsm = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')
    jax_elevation = jax_dsm.select('DSM')

    proj = jax_elevation.first().select(0).projection()
    slopeReprojected =  (jax_elevation.mosaic() \
                                .setDefaultProjection(proj)).resample('bicubic')

    # Reduce the collection with a median reducer.
    elevation = slopeReprojected.reduce(ee.Reducer.mean()).rename('elev')

    slope = ee.Terrain.slope(elevation).rename('slope')

    image = image.addBands(elevation).addBands(slope)

    return image

def normalize(image, roi):
    bandNames = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'NDVI', 'MNDWI', 'NDMI', 'NDBI', 'elev','slope']
  # Compute min and max of the image
    minDict = image.select(bandNames).reduceRegion(
        reducer = ee.Reducer.min(),
        geometry = roi,
        scale = 30,
        maxPixels=1e9,
        bestEffort=True,
        tileScale= 16)
    
    maxDict = image.select(bandNames).reduceRegion(
        reducer= ee.Reducer.max(),
        geometry= roi,
        scale= 30,
        maxPixels= 1e9,
        bestEffort= True,
        tileScale= 16)
    
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))

    normalized = image.select(bandNames).subtract(mins).divide(maxs.subtract(mins))

    return normalized.addBands(image.select('label'))


In [5]:
lPaired = lPaired.map(lambda image : add_elev_slope(image))
l_training = (lPaired.map(lambda image : normalize(image, roi)))
display(l_training)

In [6]:
l_training_random = l_training.randomColumn('random')
# Filtrer pour obtenir trois images aléatoires
l_validation = l_training_random.sort('random').limit(3)
# Créer une collection sans ces trois images
l_training = l_training.filter(ee.Filter.inList('system:index', l_validation.aggregate_array('system:index').getInfo()).Not())

# Sampling and training

In [7]:
# Fonction pour échantillonner une image
def sampling(image):
    samples = ee.Image(image).sample(
        numPixels=2000,
        #classBand='label', 
        scale=30, 
       #tileScale=2
    )
    return samples

# Itération sur l8_training pour échantillonner chaque image
samples = l_training.map(sampling).flatten()

# Ajouter une colonne aléatoire pour la répartition entraînement/validation
samples = samples.randomColumn()

# Filtrer les échantillons pour obtenir les ensembles d'entraînement et de validation
train_samples = samples.filter(ee.Filter.lte('random', 0.8))
val_samples = samples.filter(ee.Filter.gt('random', 0.8))

# Initialisation du classificateur
initial_classifier = ee.Classifier.smileRandomForest(100,variablesPerSplit=4,bagFraction=0.9)

# Entraîner le classificateur
trained_classifier = initial_classifier.train(
    features=train_samples,
    classProperty='label',
    inputProperties=['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'NDVI', 'MNDWI', 'NDMI', 'NDBI', 'elev','slope']
)


In [8]:
#display(trained_classifier.explain())

# 1rst training

In [9]:
train_confusion_matrix = trained_classifier.confusionMatrix()
val_classified = val_samples.classify(trained_classifier)
val_confusion_matrix = val_classified.errorMatrix('label', 'classification')

In [10]:
train_accuracy = train_confusion_matrix.accuracy()
val_accuracy = val_confusion_matrix.accuracy()
train_kappa = train_confusion_matrix.kappa()
val_kappa = val_confusion_matrix.kappa()
train_f1_score = train_confusion_matrix.fscore()
val_f1_score = val_confusion_matrix.fscore()
display(train_accuracy,val_accuracy,train_kappa,val_kappa,train_f1_score,val_f1_score)

## 2nd training with only bad F1 score classes

In [11]:
l_training_2 = l_training.map(lambda image : image.updateMask(image.select('label').eq(2).Or(image.select('label').eq(3)).Or(image.select('label').eq(4)).Or(image.select('label').eq(5)).Or(image.select('label').eq(6)).Or(image.select('label').eq(7))))
display(l_training_2)

In [12]:
# Fonction pour échantillonner une image
def sampling(image):
    samples = ee.Image(image).sample(
        numPixels=2000,
        #classBand='label', 
        scale=30, 
        #tileScale=2
    )
    return samples

# Itération sur l8_training pour échantillonner chaque image
samples_2 = l_training_2.map(sampling).flatten()

# Ajouter une colonne aléatoire pour la répartition entraînement/validation
samples_2 = samples_2.randomColumn()

# Filtrer les échantillons pour obtenir les ensembles d'entraînement et de validation
train_samples_2 = samples_2.filter(ee.Filter.lte('random', 0.8))
val_samples_2 = samples_2.filter(ee.Filter.gt('random', 0.8))

# Entraîner le classificateur
trained_classifier_2 = trained_classifier.train(
    features=train_samples_2,
    classProperty='label',
    inputProperties=['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'NDVI', 'MNDWI', 'NDMI', 'NDBI', 'elev','slope']
)

## Validation of 2nd training on only bad F1 score classes from 1rst training

In [13]:
train_confusion_matrix_2 = trained_classifier_2.confusionMatrix()
val_classified_2 = val_samples.classify(trained_classifier_2)
val_confusion_matrix_2 = val_classified_2.errorMatrix('label', 'classification')

In [14]:
train_accuracy_2 = train_confusion_matrix_2.accuracy()
val_accuracy_2 = val_confusion_matrix_2.accuracy()
train_kappa_2 = train_confusion_matrix_2.kappa()
val_kappa_2 = val_confusion_matrix_2.kappa()
train_f1_score_2 = train_confusion_matrix_2.fscore()
val_f1_score_2 = val_confusion_matrix_2.fscore()
display(train_accuracy_2,val_accuracy_2,train_kappa_2,val_kappa_2,train_f1_score_2,val_f1_score_2)

## Validation 

In [15]:
val_samples_3 = l_validation.map(sampling).flatten()
val_classified_3 = val_samples_3.classify(trained_classifier_2)
val_confusion_matrix_3 = val_classified_3.errorMatrix('label', 'classification')

val_accuracy_3 = val_confusion_matrix_3.accuracy()
val_kappa_3 = val_confusion_matrix_3.kappa()
val_f1_score_3 = val_confusion_matrix_3.fscore()
display(val_accuracy_3,val_kappa_3,val_f1_score_3)

# Classification

In [ ]:
min_water_date = ee.Date('2022-09-25')
max_water_date = ee.Date('2018-05-09')
wetlands = wetlands_unsupervised_clustering.getWetlandsS2(roi, min_water_date, max_water_date)

## Classification : Monthly composite

In [18]:
def add_elev_slope(image): 

    jax_dsm = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')
    jax_elevation = jax_dsm.select('DSM')

    proj = jax_elevation.first().select(0).projection()
    slopeReprojected =  (jax_elevation.mosaic() \
                                .setDefaultProjection(proj)).resample('bicubic')

    # Reduce the collection with a median reducer.
    elevation = slopeReprojected.reduce(ee.Reducer.mean()).rename('elev')

    slope = ee.Terrain.slope(elevation).rename('slope')

    image = image.addBands(elevation).addBands(slope)

    return image

def normalize(image, roi):
    bandNames = ['NDVI', 'MNDWI', 'NDBI', 'NDMI', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'elev','slope']
  # Compute min and max of the image
    minDict = image.select(bandNames).reduceRegion(
        reducer = ee.Reducer.min(),
        geometry = roi,
        scale = 30,
        maxPixels=1e9,
        bestEffort=True,
        tileScale= 16)
    
    maxDict = image.select(bandNames).reduceRegion(
        reducer= ee.Reducer.max(),
        geometry= roi,
        scale= 30,
        maxPixels= 1e9,
        bestEffort= True,
        tileScale= 16)
    
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))

    normalized = image.select(bandNames).subtract(mins).divide(maxs.subtract(mins))

    return normalized.addBands(image.select('label')).copyProperties(image,['year','month'])

def create_monthly_composites(collection):
    def add_year_month(image):
        date = ee.Date(image.get('system:time_start'))
        year = date.get('year')
        month = date.get('month')
        return image.set('year', year).set('month', month)
    
    # Ajouter les propriétés 'year' et 'month' à chaque image
    collection = collection.map(add_year_month)
    
    # Obtenir la liste des années et des mois uniques
    years = ee.List(collection.aggregate_array('year')).distinct().sort()
    months = ee.List.sequence(1, 12)
    
    # Fonction pour créer les composites mensuels
    def composite_year_month(year, month):
        year = ee.Number(year)
        month = ee.Number(month)
        filtered = collection.filter(ee.Filter.calendarRange(year, year, 'year'))\
                             .filter(ee.Filter.calendarRange(month, month, 'month'))
        composite = filtered.median()
        return composite.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1))

    # Créer les composites pour chaque année et chaque mois
    composites = years.map(lambda year: months.map(lambda month: composite_year_month(year, month))).flatten()

    # Retourner la collection de composites
    return ee.ImageCollection(composites)

def filter_non_empty_bands(collection, roi, band):
    def has_bands(image):
        # Vérifier si l'image a des bandes en comptant les noms des bandes
        band_count = image.bandNames().size()
        return image.set('band_count', band_count)
    
    # Ajouter la propriété 'has_bands' à chaque image
    collection_with_band_info = collection.map(has_bands)
    
    # Filtrer les images qui contiennent au moins une bande
    filtered_collection = collection_with_band_info.filter(ee.Filter.gt('band_count', 0))
    filter_col(filtered_collection, roi, band, 0.99)
    
    return filtered_collection

def calculate_pixel_counts(image):
    pixel_count_stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram().unweighted(),
        geometry=roi,
        scale=30,
        maxPixels=1e10
    )
    pixel_counts = ee.Dictionary(pixel_count_stats.get('classification'))
    return ee.Feature(None,pixel_counts).copyProperties(image,['month','year'])

def join_and_add_label(collection, gd_composite):
    # Créer une jointure
    join = ee.Join.inner()

    # Définir une condition de correspondance exacte des dates
    filter_time_eq = ee.Filter.equals(leftField='system:time_start', rightField='system:time_start')

    # Effectuer la jointure
    joined_collection = join.apply(primary=collection, secondary=gd_composite, condition=filter_time_eq)

    # Fonction pour fusionner les bandes de l'image originale et de l'image 'label'
    def merge_bands(joined):
        primary = ee.Image(joined.get('primary'))
        secondary = ee.Image(joined.get('secondary')).select('label')
        return ee.Image(primary).addBands(ee.Image(secondary))

    # Appliquer la fusion des bandes sur toutes les images jointes
    return joined_collection.map(merge_bands)

### Validation by testing

In [18]:
# Fonction pour échantillonner une image
def sampling(image):
    samples = ee.Image(image).sample(
        numPixels=2000,
        region = roi, 
        scale=30, 
        #tileScale=2
    )
    return samples

l7_composite_2021 = filter_non_empty_bands(create_monthly_composites(l7.filterDate('2019-01-01','2019-12-30')), roi, 'SR_B1')
l8_composite_2021 = filter_non_empty_bands(create_monthly_composites(l8.filterDate('2019-01-01','2019-12-30')), roi, 'SR_B1')
gd_composite_2021 = filter_non_empty_bands(create_monthly_composites(gd.filterDate('2019-01-01','2019-12-30')),roi,'label')

l7_validation = join_and_add_label(l7_composite_2021, gd_composite_2021).map(lambda image : normalize(add_elev_slope(ee.Image(image)),roi))
l8_validation = join_and_add_label(l8_composite_2021, gd_composite_2021).map(lambda image : normalize(add_elev_slope(ee.Image(image)),roi))

l_validation = l8_validation.merge(l7_validation)

val_samples_3 = l_validation.map(sampling).flatten()
val_classified_3 = val_samples_3.classify(trained_classifier_2)
val_confusion_matrix_3 = val_classified_3.errorMatrix('label', 'classification')

val_accuracy_3 = val_confusion_matrix_3.accuracy()
val_kappa_3 = val_confusion_matrix_3.kappa()
val_f1_score_3 = val_confusion_matrix_3.fscore()
display(val_accuracy_3,val_kappa_3,val_f1_score_3)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Collection.errorMatrix: Property 'NDMI' of feature '1_2_2_0' is missing.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Collection.errorMatrix: Property 'NDMI' of feature '1_2_2_0' is missing.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/eerepr/repr.py:57: UserWarning: Getting info failed with: 'Collection.errorMatrix: Property 'NDMI' of feature '1_2_2_0' is missing.'. Falling back to string repr.
  warn(f"Getting info failed with: '{e}'. Falling back to string repr.")


In [19]:
start_year = 1999
end_year = 2024
sensor = 'L7'

for year in range(start_year, end_year, 2):
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year + 1, 12, 31)

        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)

        # Appliquer les fonctions de traitement à la collection
        collection = create_monthly_composites(collection)
        collection = filter_non_empty_bands(collection)

        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))

        # Classifier les images[]
        classified_collection = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['year','month']))
        masked_collection = classified_collection.map(lambda image : image.updateMask(wetlands))
        # Calculer les surfaces des classes pour chaque image
        pixel_counts_fc = masked_collection.map(calculate_pixel_counts)
        # Convertir les résultats en DataFrame Pandas
        
        df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 

        df.head()

        # Ajouter des colonnes pour l'année, le mois et le type de capteur
        df['sensor'] = sensor

        # Sauvegarder les résultats dans un fichier CSV
        df.to_csv(f'landsat_{sensor}_{year}_{year+1}_wetlands.csv', index=False)

TypeError: filter_non_empty_bands() missing 2 required positional arguments: 'roi' and 'band'

In [ ]:
start_year = 2013
end_year = 2024
sensor = 'L8'

for year in range(start_year, end_year, 2):
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year + 1, 12, 31)

        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)

        # Appliquer les fonctions de traitement à la collection
        collection = create_monthly_composites(collection)
        collection = filter_non_empty_bands(collection)

        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))

        # Classifier les images
        classified_collection = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['year','month']))
        masked_collection = classified_collection.map(lambda image : image.updateMask(wetlands))
        # Calculer les surfaces des classes pour chaque image
        pixel_counts_fc = masked_collection.map(calculate_pixel_counts)
        # Convertir les résultats en DataFrame Pandas
        
        df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 

        df.head()

        # Ajouter des colonnes pour l'année, le mois et le type de capteur
        df['sensor'] = sensor

        # Sauvegarder les résultats dans un fichier CSV
        df.to_csv(f'landsat_{sensor}_{year}_{year+1}_wetlands.csv', index=False)


In [ ]:
start_year = 2021
end_year = 2024
sensor = 'L9'

for year in range(start_year, end_year, 2):
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year + 1, 12, 31)

        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.6, start_date, end_date)

        # Appliquer les fonctions de traitement à la collection
        collection = create_monthly_composites(collection)
        collection = filter_non_empty_bands(collection)

        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))

        # Classifier les images
        classified_collection = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['year','month']))
        masked_collection = classified_collection.map(lambda image : image.updateMask(wetlands))
        # Calculer les surfaces des classes pour chaque image
        pixel_counts_fc = masked_collection.map(calculate_pixel_counts)
        # Convertir les résultats en DataFrame Pandas
        
        df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 

        df.head()

        # Ajouter des colonnes pour l'année, le mois et le type de capteur
        df['sensor'] = sensor

        # Sauvegarder les résultats dans un fichier CSV
        df.to_csv(f'landsat_{sensor}_{year}_{year+1}_wetlands.csv', index=False)

## Classification : Each observations and time series

In [19]:
def add_elev_slope(image): 

    jax_dsm = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')
    jax_elevation = jax_dsm.select('DSM')

    proj = jax_elevation.first().select(0).projection()
    slopeReprojected =  (jax_elevation.mosaic() \
                                .setDefaultProjection(proj)).resample('bicubic')

    # Reduce the collection with a median reducer.
    elevation = slopeReprojected.reduce(ee.Reducer.mean()).rename('elev')

    slope = ee.Terrain.slope(elevation).rename('slope')

    image = image.addBands(elevation).addBands(slope)

    return image

def normalize(image, roi):
    bandNames = ['NDVI', 'MNDWI', 'NDBI','NDMI', 'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'elev','slope']
  # Compute min and max of the image
    minDict = image.select(bandNames).reduceRegion(
        reducer = ee.Reducer.min(),
        geometry = roi,
        scale = 30,
        maxPixels=1e9,
        bestEffort=True,
        tileScale= 16)
    
    maxDict = image.select(bandNames).reduceRegion(
        reducer= ee.Reducer.max(),
        geometry= roi,
        scale= 30,
        maxPixels= 1e9,
        bestEffort= True,
        tileScale= 16)
    
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))

    normalized = image.select(bandNames).subtract(mins).divide(maxs.subtract(mins))

    return normalized.copyProperties(image,['date'])


def filter_non_empty_bands(collection):
    def has_bands(image):
        # Vérifier si l'image a des bandes en comptant les noms des bandes
        band_count = image.bandNames().size()
        return image.set('band_count', band_count)
    
    # Ajouter la propriété 'has_bands' à chaque image
    collection_with_band_info = collection.map(has_bands)
    
    # Filtrer les images qui contiennent au moins une bande
    filtered_collection = collection_with_band_info.filter(ee.Filter.gt('band_count', 0))
    
    return filtered_collection

def calculate_pixel_counts(image):
    pixel_count_stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram().unweighted(),
        geometry=roi,
        scale=30,
        maxPixels=1e10
    )
    pixel_counts = ee.Dictionary(pixel_count_stats.get('classification'))
    return ee.Feature(None,pixel_counts).copyProperties(image,['date'])

In [ ]:
start_year = 2021
end_year = 2024
sensor = 'L9'

for year in range(start_year, end_year, 2):
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year + 1, 12, 31)

        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)

        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)

        # Classifier les images
        classified_collection = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date']))
        masked_collection = classified_collection.map(lambda image : image.updateMask(wetlands))
        # Calculer les surfaces des classes pour chaque image
        pixel_counts_fc = masked_collection.map(calculate_pixel_counts)
        # Convertir les résultats en DataFrame Pandas

        df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 

        df.head()

        # Ajouter des colonnes pour l'année, le mois et le type de capteur
        df['sensor'] = sensor
        
        # Sauvegarder les résultats dans un fichier CSV
        df.to_csv(f'landsat_{sensor}_{year}_{year+1}_each_observations.csv', index=False)

##### Landsat 8 collection

In [ ]:
start_year = 2013
end_year = 2024
sensor = 'L8'

for year in range(start_year, end_year, 2):
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year + 1, 12, 31)

        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)
        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)

        # Classifier les images
        classified_collection = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date']))
        masked_collection = classified_collection.map(lambda image : image.updateMask(wetlands))
        # Calculer les surfaces des classes pour chaque image
        pixel_counts_fc = masked_collection.map(calculate_pixel_counts)
        # Convertir les résultats en DataFrame Pandas

        df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 

        df.head()

        # Ajouter des colonnes pour l'année, le mois et le type de capteur
        df['sensor'] = sensor
        
        # Sauvegarder les résultats dans un fichier CSV
        df.to_csv(f'landsat_{sensor}_{year}_{year+1}_each_observations.csv', index=False)

##### Landsat 7 collection

In [ ]:
start_year = 1999
end_year = 2024
sensor = 'L7'

for year in range(start_year, end_year, 2):
        start_date = ee.Date.fromYMD(year, 1, 1)
        end_date = ee.Date.fromYMD(year + 1, 12, 31)

        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI'], 0.95, start_date, end_date)

        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)

        # Classifier les images
        classified_collection = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date']))
        masked_collection = classified_collection.map(lambda image : image.updateMask(wetlands))
        # Calculer les surfaces des classes pour chaque image
        pixel_counts_fc = masked_collection.map(calculate_pixel_counts)
        # Convertir les résultats en DataFrame Pandas

        df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 

        df.head()

        # Ajouter des colonnes pour l'année, le mois et le type de capteur
        df['sensor'] = sensor
        
        # Sauvegarder les résultats dans un fichier CSV
        df.to_csv(f'landsat_{sensor}_{year}_{year+1}_each_observations.csv', index=False)

# Display results

In [33]:
l8_img = ee.Image(l_training.toList(l_training.size()).get(3))
l8_cls = l8_img.classify(trained_classifier_2)
map = geemap.Map()

dw_vis = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419bdf',  # water
        '#397d49',  # trees
        '#88b053',  # grass
        '#7a87c6',  # flooded_vegetation
        '#e49635',  # crops
        '#dfc35a',  # shrub_and_scrub
        '#c4281b',  # built
        '#a59b8f',  # bare
        '#b39fe1',  # snow_and_ice
    ],
}

# Legend for Dynamic World
legend_dw = {
    'Water': '#419bdf',
    'Trees': '#397d49',
    'Grass': '#88b053',
    'Flooded Vegetation': '#7a87c6',
    'Crops': '#e49635',
    'Shrub and Scrub': '#dfc35a',
    'Built': '#c4281b',
    'Bare': '#a59b8f',
    'Snow and Ice': '#b39fe1',
}

map.centerObject(roi, 14)
# Create and add the time slider with appropriate number of labels
map.addLayer(l8_cls.select('classification'), dw_vis, 'cls')
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()
map.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
map

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [35]:
l8_img = ee.Image(l8Paired.toList(l8Paired.size()).get((3)))
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(optical_bands, None, True)

map = geemap.Map()
l8_rgbVis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.1}
map.addLayer(apply_scale_factors(l8_img), l8_rgbVis, 'true col')
map.centerObject(roi, 14)
# Create and add the time slider with appropriate number of labels
#map.addLayer(l8_cls.select('classification'), dw_vis, 'cls')

wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()
map.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
map

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

# Spatial change detection

In [28]:
start_year = 2021
end_year = 2024
sensor = 'L8'

col_1 = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI', 'NDMI'], 0.3, '2017-05-01', '2017-09-30')
col_2 = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI', 'NDMI'], 0.3, '2023-05-01', '2023-09-30')

col_1 = col_1.map(add_dates)
col_1 = col_1.map(add_elev_slope)
col_1 = col_1.map(lambda image: normalize(image, roi))
col_1= filter_non_empty_bands(col_1)

col_2 = col_2.map(add_dates)
col_2 = col_2.map(add_elev_slope)
col_2 = col_2.map(lambda image: normalize(image, roi))
col_2= filter_non_empty_bands(col_2)

        # Classifier les images
classified_col_1 = col_1.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date']))
classified_col_2 = col_2.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date']))

img_1 = classified_col_1.mode()
img_2 = classified_col_2.mode()

In [29]:
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()

In [33]:
#Entre 2017 et 2023

condition_neg = (img_1.select('classification').eq(2).Or(img_1.select('classification').eq(3))).And(img_2.select('classification').eq(5))
# Mask the image to keep only the relevant pixels
change_detection_neg = condition_neg.selfMask()

condition_pos = img_1.select('classification').eq(5).And((img_2.select('classification').eq(2)).Or(img_2.select('classification').eq(3)))
change_detection_pos = condition_pos.selfMask()

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(optical_bands, None, True)

col_1 = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'MNDWI', 'NDBI', 'NDMI'], 0.3, '2017-05-01', '2017-09-30')

# Add the change detection layer to the map
m = geemap.Map()
m.centerObject(roi, 14)
m.addLayer(apply_scale_factors(col_1.median()), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.1},'s2_img' )
m.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
m.addLayer(change_detection_neg, {'palette':['red']}, 'negative changes')
m.addLayer(change_detection_pos, {'palette':['green']}, 'positive changes')
# Display the map
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [53]:
map = geemap.Map()

dw_vis = {
    'min': 0,
    'max': 8,
    'palette': [
        '#419bdf',  # water
        '#397d49',  # trees
        '#88b053',  # grass
        '#7a87c6',  # flooded_vegetation
        '#e49635',  # crops
        '#dfc35a',  # shrub_and_scrub
        '#c4281b',  # built
        '#a59b8f',  # bare
        '#b39fe1',  # snow_and_ice
    ],
}

# Legend for Dynamic World
legend_dw = {
    'Water': '#419bdf',
    'Trees': '#397d49',
    'Grass': '#88b053',
    'Flooded Vegetation': '#7a87c6',
    'Crops': '#e49635',
    'Shrub and Scrub': '#dfc35a',
    'Built': '#c4281b',
    'Bare': '#a59b8f',
    'Snow and Ice': '#b39fe1',
}

map.centerObject(roi, 14)
# Create and add the time slider with appropriate number of labels
map.addLayer(img_1.select('classification').resample('bicubic'), dw_vis, 'cls_1')
map.addLayer(img_2.select('classification').resample('bicubic'), dw_vis, 'cls_2')
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()
map.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')
map

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

# Yearly time series derived from images classified for eache observation and .mode()

In [48]:
def calculate_pixel_counts(image):
    pixel_count_stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram().unweighted(),
        geometry=roi,
        scale=30,
        maxPixels=1e10
    )
    pixel_counts = ee.Dictionary(pixel_count_stats.get('classification'))
    return ee.Feature(None,pixel_counts).copyProperties(image,['year'])

In [49]:
start_year = 2022
end_year = 2024
sensor = 'L9'

pixel_counts_fc = ee.FeatureCollection([])

for year in range(start_year, end_year, 1):
        start_date = ee.Date.fromYMD(year, 5, 1)
        end_date = ee.Date.fromYMD(year, 9, 30)
        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)
        # Classifier les images
        classified_img = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date'])).mode()
        masked_img = classified_img.updateMask(wetlands).set('year',year)
        # Calculer les surfaces des classes pour chaque image
        pixel_counts = calculate_pixel_counts(masked_img)
        pixel_counts_fc = pixel_counts_fc.merge(ee.FeatureCollection([ee.Feature(pixel_counts)]))
        # Convertir les résultats en DataFrame Pandas


df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 
df.head()
        # Ajouter des colonnes pour l'année, le mois et le type de capteur
df['sensor'] = sensor
        # Sauvegarder les résultats dans un fichier CSV
df.to_csv(f'landsat_{sensor}_each_observations_yearly_mode.csv', index=False)

start_year = 2013
end_year = 2024
sensor = 'L8'

pixel_counts_fc = ee.FeatureCollection([])

for year in range(start_year, end_year, 1):
        start_date = ee.Date.fromYMD(year, 5, 1)
        end_date = ee.Date.fromYMD(year, 9, 30)
        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)
        # Classifier les images
        classified_img = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date'])).mode()
        masked_img = classified_img.updateMask(wetlands).set('year',year)
        # Calculer les surfaces des classes pour chaque image
        pixel_counts = calculate_pixel_counts(masked_img)
        pixel_counts_fc = pixel_counts_fc.merge(ee.FeatureCollection([ee.Feature(pixel_counts)]))
        # Convertir les résultats en DataFrame Pandas


df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 
df.head()
        # Ajouter des colonnes pour l'année, le mois et le type de capteur
df['sensor'] = sensor
        # Sauvegarder les résultats dans un fichier CSV
df.to_csv(f'landsat_{sensor}_each_observations_yearly_mode.csv', index=False)

In [51]:
start_year = 1999
end_year = 2005
sensor = 'L7'

pixel_counts_fc = ee.FeatureCollection([])

for year in range(start_year, end_year, 1):
        start_date = ee.Date.fromYMD(year, 5, 1)
        end_date = ee.Date.fromYMD(year, 9, 30)
        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)
        # Classifier les images
        classified_img = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date'])).mode()
        masked_img = classified_img.updateMask(wetlands).set('year',year)
        # Calculer les surfaces des classes pour chaque image
        pixel_counts = calculate_pixel_counts(masked_img)
        pixel_counts_fc = pixel_counts_fc.merge(ee.FeatureCollection([ee.Feature(pixel_counts)]))
        # Convertir les résultats en DataFrame Pandas


df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 
df.head()
        # Ajouter des colonnes pour l'année, le mois et le type de capteur
df['sensor'] = sensor
        # Sauvegarder les résultats dans un fichier CSV
df.to_csv(f'landsat_{sensor}_each_observations_yearly_mode_1999_2005.csv', index=False)


start_year = 2006
end_year = 2012
sensor = 'L7'

pixel_counts_fc = ee.FeatureCollection([])

for year in range(start_year, end_year, 1):
        start_date = ee.Date.fromYMD(year, 5, 1)
        end_date = ee.Date.fromYMD(year, 9, 30)
        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)
        # Classifier les images
        classified_img = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date'])).mode()
        masked_img = classified_img.updateMask(wetlands).set('year',year)
        # Calculer les surfaces des classes pour chaque image
        pixel_counts = calculate_pixel_counts(masked_img)
        pixel_counts_fc = pixel_counts_fc.merge(ee.FeatureCollection([ee.Feature(pixel_counts)]))
        # Convertir les résultats en DataFrame Pandas


df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 
df.head()
        # Ajouter des colonnes pour l'année, le mois et le type de capteur
df['sensor'] = sensor
        # Sauvegarder les résultats dans un fichier CSV
df.to_csv(f'landsat_{sensor}_each_observations_yearly_mode_2006_2012.csv', index=False)

EEException: Too many concurrent aggregations.

In [52]:
start_year = 2013
end_year = 2018
sensor = 'L7'

pixel_counts_fc = ee.FeatureCollection([])

for year in range(start_year, end_year, 1):
        start_date = ee.Date.fromYMD(year, 5, 1)
        end_date = ee.Date.fromYMD(year, 9, 30)
        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)
        # Classifier les images
        classified_img = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date'])).mode()
        masked_img = classified_img.updateMask(wetlands).set('year',year)
        # Calculer les surfaces des classes pour chaque image
        pixel_counts = calculate_pixel_counts(masked_img)
        pixel_counts_fc = pixel_counts_fc.merge(ee.FeatureCollection([ee.Feature(pixel_counts)]))
        # Convertir les résultats en DataFrame Pandas


df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 
df.head()
        # Ajouter des colonnes pour l'année, le mois et le type de capteur
df['sensor'] = sensor
        # Sauvegarder les résultats dans un fichier CSV
df.to_csv(f'landsat_{sensor}_each_observations_yearly_mode_2013_2018.csv', index=False)

start_year = 2018
end_year = 2024
sensor = 'L7'

pixel_counts_fc = ee.FeatureCollection([])

for year in range(start_year, end_year, 1):
        start_date = ee.Date.fromYMD(year, 5, 1)
        end_date = ee.Date.fromYMD(year, 9, 30)
        # Sélectionner la collection Landsat appropriée
        if sensor == 'L7':
            collection = landsat_preprocessing.get_l7_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L8':
            collection = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        elif sensor == 'L9':
            collection = landsat_preprocessing.get_l9_cloud_free_col_dates(roi, ['NDVI', 'NDMI', 'MNDWI', 'NDBI'], 0.3, start_date, end_date)
        # Appliquer les fonctions de traitement à la collection
        collection = collection.map(add_dates)
        collection = collection.map(add_elev_slope)
        collection = collection.map(lambda image: normalize(image, roi))
        collection = filter_non_empty_bands(collection)
        # Classifier les images
        classified_img = collection.map(lambda image: image.classify(trained_classifier_2).copyProperties(image,['date'])).mode()
        masked_img = classified_img.updateMask(wetlands).set('year',year)
        # Calculer les surfaces des classes pour chaque image
        pixel_counts = calculate_pixel_counts(masked_img)
        pixel_counts_fc = pixel_counts_fc.merge(ee.FeatureCollection([ee.Feature(pixel_counts)]))
        # Convertir les résultats en DataFrame Pandas


df =  ee.data.computeFeatures({'expression': pixel_counts_fc,'fileFormat': 'PANDAS_DATAFRAME'}) 
df.head()
        # Ajouter des colonnes pour l'année, le mois et le type de capteur
df['sensor'] = sensor
        # Sauvegarder les résultats dans un fichier CSV
df.to_csv(f'landsat_{sensor}_each_observations_yearly_mode_2018_2024.csv', index=False)

# 